<a href="https://colab.research.google.com/github/nzgnzg73/Voice-Clone-Kaggle-Google-Colab/blob/main/ChatterboxToolkitUI_(Google_Coloab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tech NZG**

**Suscribe Button ✅ click 😊😊**

**https://youtube.com/@hubnzg?si=mbIuCBwmgAwoCrYJ**

**Setp (1)**

In [ ]:
# @title Default title text
# ------------------------------------------------------------------
# 1️⃣  System packages (ffmpeg) – needed by pydub / many TTS back-ends
# ------------------------------------------------------------------
!apt-get -qq update && apt-get -qq install -y ffmpeg

# ------------------------------------------------------------------
# 2️⃣  Python packages
# ------------------------------------------------------------------
!pip -q install --upgrade pip
!pip -q install torch torchvision==0.0+cu118
!pip -q uninstall -y torchvision
!pip -q install chatterbox-tts gradio pydub scipy webrtcvad pyngrok

# ------------------------------------------------------------------
# 2️⃣  Clone ChatterboxToolkitUI
# ------------------------------------------------------------------
!git clone https://github.com/dasjoms/ChatterboxToolkitUI.git
%cd ChatterboxToolkitUI

# ------------------------------------------------------------------
# 3️⃣  Patch launch call
# ------------------------------------------------------------------
import pathlib, re

ui_path = pathlib.Path("ChatterboxToolkitUI.py")
code = ui_path.read_text(encoding="utf-8")

patched = re.sub(r"demo\.queue\(\)\.launch\(\)",
                "demo.queue().launch(share=False)",
                code, count=1)

# 🔧 آڈیو آؤٹ پٹ میں type="filepath" ایڈ کریں
patched = patched.replace('gr.Audio(', 'gr.Audio(type="filepath", ')

ui_path.write_text(patched, encoding="utf-8")
print("✅ Patched launch() → share=False + audio download enabled")

# ------------------------------------------------------------------
# 4️⃣  Run the UI
# ------------------------------------------------------------------
import subprocess, os, signal, sys, pathlib

log_file = pathlib.Path("ui.log")

ui_proc = subprocess.Popen(
    ["python", "ChatterboxToolkitUI.py"],
    stdout=log_file.open("w", buffering=1),
    stderr=subprocess.STDOUT,
    preexec_fn=os.setsid
)

print(f"🚀 UI started (PID={ui_proc.pid}). Logs → {log_file}")
print("🖥️  You can watch them live with:  !tail -f ui.log")



**Setp (2)**

Google Colab Notebook - AI Tool To use this notebook, please follow these steps:

Click on the provided link below to open the Google Colab file.
In the code cell, add your personal ngrok authentication token.
If you don't have a token, you can get one for free from the official ngrok website: Ngrok Website: https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# @title Default title text
# --------------------------------------------------------------
#  ngrok ٹوکن – اپنی ٹوکن یہاں رکھیں
# --------------------------------------------------------------
NGROK_TOKEN = "31lm4MJfVdJcgD1q7VVnpDcAo47_5f41XeCDHXeZRHFoRWi7r"

if NGROK_TOKEN:
    from pyngrok import conf
    conf.get_default().auth_token = NGROK_TOKEN
    print("✅ ngrok کا ٹوکن سیٹ ہوا")
else:
    raise ValueError("ngrok ٹوکن مہیا نہیں کیا گیا")


**Setp (3)**

In [ ]:
# @title Default title text
# --------------------------------------------------------------
# ✅  Dependencies
# --------------------------------------------------------------
!apt-get -qq update && apt-get -qq install -y ffmpeg > /dev/null
!pip -q install chatterbox-tts gradio pydub scipy webrtcvad pyngrok > /dev/null

# --------------------------------------------------------------
# ✅  Clone the repo (fresh copy, no old patches)
# --------------------------------------------------------------
!rm -rf ChatterboxToolkitUI
!git clone https://github.com/dasjoms/ChatterboxToolkitUI.git
%cd ChatterboxToolkitUI

# --------------------------------------------------------------
# ✅  Patch launch + fix audio download (only for Output audio)
# --------------------------------------------------------------
import pathlib, re

ui_path = pathlib.Path("ChatterboxToolkitUI.py")
code = ui_path.read_text(encoding="utf-8")

# patch launch → disable share=True
new_code = re.sub(
    r"demo\.queue\(\)\.launch\(\)",
    "demo.queue().launch(share=False)",
    code,
    count=1
)

# ✅ Patch only OUTPUT audio widgets (label contains 'Output')
def patch_output_audio(match):
    widget = match.group(0)
    # replace existing type=... with filepath
    if "type=" in widget:
        widget = re.sub(r'type\s*=\s*"[a-zA-Z0-9_]+"', 'type="filepath"', widget)
    else:
        widget = widget.replace("gr.Audio(", 'gr.Audio(type="filepath", ')
    return widget

new_code = re.sub(
    r'gr\.Audio\([^)]*label\s*=\s*"[^"]*Output[^"]*"[^)]*\)',
    patch_output_audio,
    new_code
)

ui_path.write_text(new_code, encoding="utf-8")
print("✅ demo.queue().launch patched + audio download enabled for OUTPUT only")

# --------------------------------------------------------------
# ✅ Run the UI in background
# --------------------------------------------------------------
import subprocess, os, signal, time

ui_process = subprocess.Popen(
    ["python", "ChatterboxToolkitUI.py"],
    stdout=open("ui.log", "w"),
    stderr=subprocess.STDOUT,
    preexec_fn=os.setsid
)

print("🚀 UI started (PID = %d) – logs are in ui.log" % ui_process.pid)

time.sleep(20)

# quick check
import requests
try:
    r = requests.get("http://127.0.0.1:7860")
    print("✅ UI running, status:", r.status_code)
except Exception as e:
    print("⚠️ UI not yet reachable:", e)

# --------------------------------------------------------------
# ✅ ngrok tunnel
# --------------------------------------------------------------
from pyngrok import ngrok, exception as ngrok_exc

NGROK_TOKEN = "31AgTx1Om0HtkOBFz9cPqVLQF63_7ZgDC7ZhEChNWfkiMcgEt"
if NGROK_TOKEN:
    from pyngrok import conf
    conf.get_default().auth_token = NGROK_TOKEN
    print("✅ ngrok token set")
else:
    raise ValueError("ngrok token missing")

ngrok.kill()

def start_ngrok(port: int = 7860):
    try:
        tunnel = ngrok.connect(addr=str(port), bind_tls=True)
        print("\n🔗 ngrok public URL :", tunnel.public_url)
        return tunnel.public_url
    except ngrok_exc.PyngrokError as e:
        raise RuntimeError(f"ngrok failed : {e}")

public_url = start_ngrok(port=7860)

print("\n🖥️  Local URL (inside Colab) : http://127.0.0.1:7860")
print("\n🔁 اگر ٹنل بند ہو جائے تو اس سیل دوبارہ چلائیں۔")


**Before closing Google Colab, you must run this final command to shut down the ngrok session. If you do not close it, your token could become unsecured and is at risk of being hacked. Please take special care of this.**

**👇🏼👇🏼👇🏼👇🏼👇🏼👇🏼👇🏼👇🏼👇🏼👇🏼😱**

In [ ]:
from pyngrok import ngrok

# This command will completely shut down all ngrok processes

ngrok.kill()